In [ ]:
!pip install tensorflow-addons
!pip install pyyaml h5py
!pip install toolz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 3.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
Tensorflow version 2.9.2


## Setup

In [ ]:
import os
# os.environ["MODIN_CPUS"] = "8"
# from distributed import Client
# client = Client()
import numpy as np
import math
import re
import random
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.backend as K
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow_addons as tfa
from sklearn import metrics
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import constraints
from tensorflow.keras import initializers
from tensorflow.keras import regularizers
from tensorflow.keras.applications import efficientnet as efn
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from tensorflow.keras.constraints import Constraint
# import allel
from scipy.spatial.distance import squareform
%matplotlib inline   
from toolz import interleave
from tqdm import tqdm
from matplotlib import pyplot as plt
import tensorflow_datasets as tfds
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LassoCV, ElasticNetCV
from sklearn.model_selection import KFold,StratifiedKFold

print("Tensorflow version " + tf.__version__)

Tensorflow version 2.9.2


## Hardware Config

In [ ]:
# Detect hardware, return appropriate distribution strategy
try:
    TPU = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    print('Running on TPU ', TPU.master())
except ValueError:
    print('Running on GPU')
    TPU = None

if TPU:
    tf.config.experimental_connect_to_cluster(TPU)
    tf.tpu.experimental.initialize_tpu_system(TPU)
    strategy = tf.distribute.TPUStrategy(TPU)
else:
    strategy = tf.distribute.get_strategy() # default distribution strategy in Tensorflow. Works on CPU and single GPU.

N_REPLICAS = strategy.num_replicas_in_sync
# Number of computing cores, is 8 for a TPU V3-8
print(f'N_REPLICAS: {N_REPLICAS}')

Running on GPU
N_REPLICAS: 1


## Prepare the data

In [ ]:
# load genotype
genotypes = pd.read_csv('[path]/genotype_full.txt', sep='\t', index_col=0)
genotypes[genotypes == -1] = 0

headers = genotypes.columns[:]

In [ ]:
genotypes.head()

,33070_chrI_33070_A_T,33147_chrI_33147_G_T,33152_chrI_33152_T_C,33200_chrI_33200_C_T,33293_chrI_33293_A_T,33328_chrI_33328_C_A,33348_chrI_33348_G_C,33403_chrI_33403_C_T,33502_chrI_33502_A_G,33548_chrI_33548_A_C,...,12048853_chrXVI_925593_G_C,12049199_chrXVI_925939_T_C,12049441_chrXVI_926181_C_T,12050613_chrXVI_927353_T_G,12051167_chrXVI_927907_A_C,12051240_chrXVI_927980_A_G,12051367_chrXVI_928107_C_T,12052782_chrXVI_929522_C_T,12052988_chrXVI_929728_A_G,12053130_chrXVI_929870_C_T
SAMID,,,,,,,,,,,,,,,,,,,,,
01_01,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
01_02,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
01_03,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
01_04,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
01_06,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
X = genotypes
X.shape

(4390, 28220)

In [ ]:
# hyperparameters
feature_size = X.shape[1]
inChannel = 3
# optimizer learning rate
learning_rate = 0.001
epochs = 60 
#epochs = 100   # chr20 LOS 5K


# training batch size
#batch_size = 32   # u19, 4984 samples
bs = 48


lr = 1e-3



# l1 regulalization
kr = 1e-4
k_initial = 'glorot_uniform'


kernel_len = 5

# channel = inChannel
channel = inChannel

num_latent = 128
p_size = 2
ndf_num = 32


#dr_rate = drop_prec
dr_rate = 0.2  # avoid overfitting for missing ratio of 0.7

## Model

In [ ]:
# AE model in one cell



# # V1: verify our own ae model with yeast genotype data

# # 2.2.8 build variatial autoencoder for snp with subclassing function. 

class SNP_ENCODER(tf.keras.Model):    
    def __init__(self, feature_size, channel=channel, ndf=ndf_num, n_latent=num_latent, dr=dr_rate):    
        super(SNP_ENCODER, self).__init__()
        self.feature_size = feature_size
        self.channel = channel
        self.ndf = ndf
        self.n_latent = n_latent
        self.dr = dr # dropout rate

        # object, can be saved in tf mode
        self.stride=1
        self.kl = 5

        
    def build(self, inputs): 
        #encoder
        # dense layer 1
        self.c1 = layers.Conv1D(filters=self.ndf, kernel_size=self.kl, strides=self.stride, padding="same", 
                      activation='relu', use_bias=True, 
                      kernel_initializer=k_initial, kernel_regularizer=tf.keras.regularizers.L1(kr), 
                      input_shape=(self.feature_size, self.channel))
        self.p1 = layers.MaxPooling1D(pool_size=p_size)
        self.drop1 = layers.Dropout(rate=self.dr)
        
        # dense layer 2
        self.c2 = layers.Conv1D(filters=(2*self.ndf), kernel_size=self.kl, strides=self.stride, padding="same", 
                      activation='relu', use_bias=True, kernel_initializer=k_initial, 
                      kernel_regularizer=tf.keras.regularizers.L1(kr))
        self.p2 = layers.MaxPooling1D(pool_size=p_size)
        self.drop2 = layers.Dropout(rate=self.dr)
        
        # dense layer 3
        self.c3 = layers.Conv1D(filters=(4*self.ndf), kernel_size=self.kl, strides=1, padding="same", 
                      activation='relu', use_bias=True, kernel_initializer=k_initial, 
                      kernel_regularizer=tf.keras.regularizers.L1(kr))
        self.p3 = layers.MaxPooling1D(pool_size=p_size)
        self.drop3 = layers.Dropout(rate=self.dr)

        super(SNP_ENCODER, self).build(inputs)
    
    def call(self, inputs, training=True):
        #print('SNP_ENCODER training flag: ', training)
        x = self.c1(inputs)
        x = self.p1(x)
        x = self.drop1(x, training=training)

        x = self.c2(x)
        x = self.p2(x)
        x = self.drop2(x, training=training)
        
        x = self.c3(x)
        return x

    # AFAIK: The most convenient method to print model.summary() 
    # similar to the sequential or functional API like.
    def build_graph(self):
        x = layers.Input(shape=(self.feature_size, self.channel))
        return tf.keras.Model(inputs=[x], outputs=self.call(x))
    
    
    
# SNP_DECODER(keras.Model):   
class SNP_DECODER(tf.keras.Model):     
    def __init__(self, feature_size, channel=channel, ndf=ndf_num, n_latent=num_latent, dr=dr_rate):
        super(SNP_DECODER, self).__init__()
        self.feature_size = feature_size
        self.channel = channel
        self.ndf = ndf
        self.n_latent = n_latent
        self.dr = dr # dropout rate
        
        # object, can be saved in tf mode
        self.stride=1
        self.kl=5
        
    def build(self, inputs):
        #decoder        
        self.c1 = layers.Conv1D(filters=(2*self.ndf), kernel_size=self.kl, strides=self.stride, padding="same", 
                      activation='relu', use_bias=True, 
                      kernel_initializer=k_initial, kernel_regularizer=tf.keras.regularizers.L1(kr),
                      input_shape=((self.feature_size>>2), self.n_latent))
        
        
        self.s1 = layers.UpSampling1D(size=p_size)
        self.drop1 = layers.Dropout(rate=self.dr)
        
        # dense layer 2
        self.c2 = layers.Conv1D(filters=(1*self.ndf), kernel_size=self.kl, strides=self.stride, padding="same", 
                      activation='relu', use_bias=True, 
                      kernel_initializer=k_initial, kernel_regularizer=tf.keras.regularizers.L1(kr))

        self.s2 = layers.UpSampling1D(size=p_size)
        self.drop2 = layers.Dropout(rate=self.dr)
        

        # dense layer6
        self.c3 = layers.Conv1D(filters=self.channel, kernel_size=self.kl, strides=1, padding="same", 
                      activation='softmax', use_bias=True)
        
        super(SNP_DECODER, self).build(inputs)
        
    def call(self, inputs, training = True):
        #print('SNP_DECODER training flag: ', training)
        x = self.c1(inputs)
        x = self.s1(x)
        x = self.drop1(x, training=training)
        
        x = self.c2(x)
        x = self.s2(x)
        x = self.drop2(x, training=training)

        
        d_out = self.c3(x)
        return d_out
    
    # AFAIK: The most convenient method to print model.summary() 
    # similar to the sequential or functional API like.
    def build_graph(self):
        x = layers.Input(shape=(self.feature_size>>2, self.n_latent))
        
        return tf.keras.Model(inputs=[x], outputs=self.call(x))
    

    
#class SNP_AE(keras.Model):
class SNP_AE(tf.keras.Model):    
    def __init__(self, feature_size, channel=channel, ndf=ndf_num, n_latent=num_latent, dr=dr_rate):    
        super(SNP_AE, self).__init__()
        self.feature_size = feature_size
        self.channel = channel
        self.ndf = ndf
        self.n_latent = n_latent
        self.dr = dr # dropout rate      
        
        self.encoder = SNP_ENCODER(self.feature_size)
        self.decoder = SNP_DECODER(self.feature_size)
    

    def call(self, x, training=True): 
        latent = self.encoder(x, training)     
        res = self.decoder(latent, training)

        return res, latent
    
    # AFAIK: The most convenient method to print model.summary() 
    # similar to the sequential or functional API like.
    def build_graph(self):
        x = layers.Input(shape=(self.feature_size, self.channel))
        return tf.keras.Model(inputs=[x], outputs=self.call(x))
    

In [ ]:
# plot snp vae encoder model

SNP_encoder = SNP_ENCODER(feature_size)
SNP_encoder.build((None, feature_size, channel))
SNP_encoder.build_graph().summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28220, 3)]        0         
                                                                 
 conv1d (Conv1D)             (None, 28220, 32)         512       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 14110, 32)        0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 14110, 32)         0         
                                                                 
 conv1d_1 (Conv1D)           (None, 14110, 64)         10304     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 7055, 64)         0         
 1D)                                                         

In [ ]:
# plot snp vae decoder model

SNP_decoder = SNP_DECODER(feature_size)

SNP_decoder.build((None, feature_size>>2, num_latent))
SNP_decoder.build_graph().summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 7055, 128)]       0         
                                                                 
 conv1d_3 (Conv1D)           (None, 7055, 64)          41024     
                                                                 
 up_sampling1d (UpSampling1D  (None, 14110, 64)        0         
 )                                                               
                                                                 
 dropout_3 (Dropout)         (None, 14110, 64)         0         
                                                                 
 conv1d_4 (Conv1D)           (None, 14110, 32)         10272     
                                                                 
 up_sampling1d_1 (UpSampling  (None, 28220, 32)        0         
 1D)                                                       

In [ ]:
# plot snp vae model

SNP_ae = SNP_AE(feature_size)
SNP_ae.build((None, feature_size, channel))
SNP_ae.build_graph().summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 28220, 3)]        0         
                                                                 
 snp_encoder_1 (SNP_ENCODER)  (None, 7055, 128)        51904     
                                                                 
 snp_decoder_1 (SNP_DECODER)  (None, 28220, 3)         51779     
                                                                 
Total params: 103,683
Trainable params: 103,683
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# AE model sub-functions

def generate_fake_missing(x_in, missing_ratio=0.5):

        # Generates missing genotypes
        # different missing loci for each individuals
        x_fake = x_in.copy()   # with .copy() to not overwrite the original data
        
        for i in range(x_in.shape[0]):
            missing_size = int(missing_ratio * x_in.shape[1])
          
            # without repeat random numbers: set replace with false
            missing_index = np.random.choice(x_in.shape[1], size=missing_size, replace=False)
            
            # missing loci are encoded as [0, 0]
            x_fake[i, missing_index, :] = [0, 0, 1]  # yeast
            # x_fake[i, missing_index, :] = [1, 0, 0, 0]  # human

        return x_fake
        #return x_fake, x_in
    
    

def loss_function_cce(recon_x, x):
    # orders: y_true, y_pred
    cce = tf.keras.losses.categorical_crossentropy(x, recon_x)

    #cce = np.double(cce)
    cce = K.cast(cce, dtype='float32')

    lamb1 = 1.0
    loss = lamb1*cce
    #print('loss:', loss)
    
    loss = tf.reduce_mean(loss)
    #print('ave loss:', loss)
    
    return loss


## Training

In [ ]:
N_SPLITS=3
results = None
kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=2022)
fold = 0
_x = tf.keras.utils.to_categorical(X.to_numpy(), 3)
print(_x.shape)
for train_index, test_index in kf.split(_x):
  fold += 1
  # if fold < 3:
  #   continue
  accuracies = []
  print(f"Training using fold {fold}")
  print("*******************************************")
  print("*******************************************")
  
  x_train = _x[train_index]
  test_dataset = _x[test_index]
  x_train, x_valid = train_test_split(x_train, test_size=0.10,
                                      random_state=fold,
                                      shuffle=True,)
   # fake missing values

  for missing_ratio in [
                        0.01,
                        0.05,
                        0.1,
                        0.2
                        ]:
    train_X = np.copy(x_train)
    valid_X = np.copy(x_valid)
    print(f"Missing rate {missing_ratio}")
    print("=====================================================")
    train_X_fake = generate_fake_missing(train_X, missing_ratio)

    diff = np.absolute(np.array(train_X) - np.array(train_X_fake))
    print('train_X_fake diff:', np.sum(diff))


    valid_X_fake = generate_fake_missing(valid_X, missing_ratio)
    diff = np.absolute(np.array(valid_X) - np.array(valid_X_fake))
    print('valid_X_fake diff:', np.sum(diff))


    K.clear_session()
    with strategy.scope():
      model = SNP_AE(feature_size)
      optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

      train_loss_metric = tf.keras.metrics.CategoricalCrossentropy()
      val_loss_metric = tf.keras.metrics.CategoricalCrossentropy()

      train_acc_metric = tf.keras.metrics.CategoricalAccuracy()
      val_acc_metric = tf.keras.metrics.CategoricalAccuracy()

      for epoch in range(epochs):
        print('Start of epoch %d' % (epoch,))

        #----shuffle train data and lablel for each epoch-----------------------------------------------------------------------------------#    
        # shuffle data and labels at the same time
        idx = train_X.shape[0]


        indices = tf.range(start=0, limit=idx, dtype=tf.int32)

        shuffled_indices = tf.random.shuffle(indices)  
      
        train_X_fake = tf.gather(train_X_fake, shuffled_indices)
        train_X = tf.gather(train_X, shuffled_indices)        
        
        snp_x = tf.data.Dataset.from_tensor_slices(train_X_fake).batch(bs, drop_remainder=True) 
        snp_y = tf.data.Dataset.from_tensor_slices(train_X).batch(bs, drop_remainder=True)

        snp_x_v = tf.data.Dataset.from_tensor_slices(valid_X_fake).batch(bs, drop_remainder=True)
        snp_y_v = tf.data.Dataset.from_tensor_slices(valid_X).batch(bs, drop_remainder=True)

        loss_batch = []

        # Iterate over the batches of the dataset.
        for step, (snp_fake_batch, snp_label_batch) in enumerate(zip(snp_x, snp_y)): 
          with tf.GradientTape() as tape:
              recon_inputs, latents= model(snp_fake_batch, training=True)
              loss = loss_function_cce(recon_inputs, snp_label_batch)


          grads = tape.gradient(loss, model.trainable_variables)
          optimizer.apply_gradients(zip(grads, model.trainable_variables))    

          loss_batch.append(loss.numpy())


          # Update training metric.
          train_acc_metric.update_state(snp_label_batch, recon_inputs)
          train_loss_metric.update_state(snp_label_batch, recon_inputs)


        # Display metrics at the end of each epoch.
        train_loss = train_loss_metric.result()
        print("Training loss over epoch: ", epoch, train_loss.numpy())

        train_acc = train_acc_metric.result()
        print("Training acc over epoch: ", epoch, train_acc.numpy())


        # Reset training metrics at the end of each epoch
        train_loss_metric.reset_states()
        train_acc_metric.reset_states()


        # Run a validation loop at the end of each epoch.
        for x_batch_val, y_batch_val in zip(snp_x_v, snp_y_v):   

            val_recons, latents = model(x_batch_val, training=False)
            # Update val metrics
            val_loss_metric.update_state(y_batch_val, val_recons)
            val_acc_metric.update_state(y_batch_val, val_recons)

        val_loss = val_loss_metric.result()
        val_acc = val_acc_metric.result()

        val_loss_metric.reset_states()
        val_acc_metric.reset_states()
        #print("Validation acc: %.4f" % (float(val_acc),))
        print("Validation loss: ", epoch, val_loss.numpy())
        print("Validation acc: ", epoch, val_acc.numpy())


        #print('epoch %s: batch loss = %s' % (epoch, loss_batch))
        loss_epoch = np.mean(loss_batch)    
        print('epoch %s: loss = %s' % (epoch, loss_epoch))    
    
    save_name = f"[path]/YEAST/AE/preds_mixed_mr_{missing_ratio}_fold_{fold}_.csv"
    avg_accuracy = []
    preds = []
    true_labels = []
    
    to_save_array = np.zeros((test_dataset.shape[0], test_dataset.shape[1]), dtype=object)
    test_X_missing = np.copy(test_dataset)
    # test_X_missing = to_categorical(test_X_missing, 3)
    for i in tqdm(list(range(test_dataset.shape[0]))):
      rng = np.random.default_rng(i + fold)
      # Generates missing genotypes
      missing_size = int(missing_ratio * test_X_missing.shape[1])
      missing_index = rng.integers(low=0, high=test_X_missing.shape[1],
                                        size=missing_size)
      test_X_missing[i:i+1, missing_index, :] = [0, 0, 1]
      # mMask = np.tile(attention_mask, (1, 1, 1))
      # predict
      predict_onehot, _ = model(test_X_missing[i:i+1, :, :], training=False)
      predict_onehot = predict_onehot.numpy()
      # predict_onehot = model.predict(test_X_missing[i:i+1, :, :])
      # predict_onehot = model.predict([test_X_missing[i:i + 1, :, :], mMask])
      # only care the missing position
      predict_missing_onehot = predict_onehot[0:1, missing_index, :]
      # predict label
      predict_missing = np.argmax(predict_missing_onehot, axis=2)
      
      preds.extend(predict_missing.ravel().tolist())
      
      predict_haplotypes = np.argmax(predict_onehot, axis=2)
      to_save_array[i] = np.where(predict_haplotypes==0, -1, 1)
      # real label
      label_missing_onehot = np.argmax(test_dataset[i:i + 1, missing_index], axis=2)
      # label_missing = np.argmax(label_missing_onehot, axis=2)
      label_missing = np.argmax(test_dataset[i:i + 1, missing_index], axis=2)
      true_labels.extend(label_missing.ravel().tolist())
      # accuracy
      correct_prediction = np.equal(predict_missing, label_missing)
      accuracy = np.mean(correct_prediction)
      # print('{}/{}, accuracy: {:.4f}'.format(
      #     i, test_X_missing.shape[0], accuracy))

      avg_accuracy.append(accuracy)

    df = pd.DataFrame(to_save_array, columns= headers[:], index = test_index)
    df.to_csv(save_name)
    print('The average imputation accuracy' \
          'on test data with {} missing genotypes is {:.4f}: '
        .format(missing_ratio, np.mean(avg_accuracy)))
    cnf_matrix = confusion_matrix(true_labels, preds)
    FP = cnf_matrix.sum(axis=0) - np.diag(cnf_matrix)
    FN = cnf_matrix.sum(axis=1) - np.diag(cnf_matrix)
    TP = np.diag(cnf_matrix)
    TN = cnf_matrix.sum() - (FP + FN + TP)
    FP = FP.astype(float)
    FN = FN.astype(float)
    TP = TP.astype(float)
    TN = TN.astype(float)
    # Sensitivity, hit rate, recall, or true positive rate
    TPR = TP/(TP+FN)
    # Specificity or true negative rate
    TNR = TN/(TN+FP)
    print(f"Sensitivity: {np.mean(TPR)}")
    print(f"Specificity: {np.mean(TNR)}")
    print(f"F1-score macro: {f1_score(true_labels, preds, average='macro')}")
    print(f"F1-score micro: {f1_score(true_labels, preds, average='micro')}")
    accuracies.append(np.mean(avg_accuracy))
      
    

(4390, 28220, 3)
Training using fold 1
*******************************************
*******************************************
Missing rate 0.01
train_X_fake diff: 1485012.0
valid_X_fake diff: 165252.0
Start of epoch 0
Training loss over epoch:  0 0.1003256
Training acc over epoch:  0 0.9660834
Validation loss:  0 0.0036228383
Validation acc:  0 0.99967515
epoch 0: loss = 0.100325614
Start of epoch 1
Training loss over epoch:  1 0.0049622436
Training acc over epoch:  1 0.9985878
Validation loss:  1 0.0023706476
Validation acc:  1 0.9997801
epoch 1: loss = 0.004962244
Start of epoch 2
Training loss over epoch:  2 0.002882032
Training acc over epoch:  2 0.9990136
Validation loss:  2 0.0010421426
Validation acc:  2 0.99989456
epoch 2: loss = 0.002882032
Start of epoch 3
Training loss over epoch:  3 0.0018415019
Training acc over epoch:  3 0.9993318
Validation loss:  3 0.0006754789
Validation acc:  3 0.99991286
epoch 3: loss = 0.0018415018
Start of epoch 4
Training loss over epoch:  4 0.00

100%|██████████| 1464/1464 [04:31<00:00,  5.39it/s]


The average imputation accuracyon test data with 0.01 missing genotypes is 0.9981: 
Sensitivity: 0.9980838606521286
Specificity: 0.9980838606521286
F1-score macro: 0.9980840345427302
F1-score micro: 0.9980840406154323
Missing rate 0.05
train_X_fake diff: 7430326.0
valid_X_fake diff: 826846.0
Start of epoch 0
Training loss over epoch:  0 0.12006121
Training acc over epoch:  0 0.9651518
Validation loss:  0 0.0044546723
Validation acc:  0 0.99928784
epoch 0: loss = 0.12006116
Start of epoch 1
Training loss over epoch:  1 0.006255433
Training acc over epoch:  1 0.9982819
Validation loss:  1 0.0029996368
Validation acc:  1 0.9995919
epoch 1: loss = 0.006255433
Start of epoch 2
Training loss over epoch:  2 0.003930305
Training acc over epoch:  2 0.99866503
Validation loss:  2 0.0018993831
Validation acc:  2 0.99972266
epoch 2: loss = 0.0039303047
Start of epoch 3
Training loss over epoch:  3 0.002857718
Training acc over epoch:  3 0.9989776
Validation loss:  3 0.0013543367
Validation acc:  3

100%|██████████| 1464/1464 [04:30<00:00,  5.41it/s]


The average imputation accuracyon test data with 0.05 missing genotypes is 0.9981: 
Sensitivity: 0.9981044969631339
Specificity: 0.9981044969631339
F1-score macro: 0.9981042782119769
F1-score micro: 0.9981042782509014
Missing rate 0.1
train_X_fake diff: 14860652.0
valid_X_fake diff: 1653692.0
Start of epoch 0
Training loss over epoch:  0 0.093389906
Training acc over epoch:  0 0.97832876
Validation loss:  0 0.0041354722
Validation acc:  0 0.9992387
epoch 0: loss = 0.0933899
Start of epoch 1
Training loss over epoch:  1 0.005161372
Training acc over epoch:  1 0.9983644
Validation loss:  1 0.0025475987
Validation acc:  1 0.99950147
epoch 1: loss = 0.0051613715
Start of epoch 2
Training loss over epoch:  2 0.0031439723
Training acc over epoch:  2 0.9988433
Validation loss:  2 0.0015263876
Validation acc:  2 0.9996608
epoch 2: loss = 0.0031439725
Start of epoch 3
Training loss over epoch:  3 0.0022235871
Training acc over epoch:  3 0.9991576
Validation loss:  3 0.0010631755
Validation acc:

...

The average imputation accuracyon test data with 0.05 missing genotypes is 0.9980: 
Sensitivity: 0.9980270521464919
Specificity: 0.9980270521464919
F1-score macro: 0.9980249702198889
F1-score micro: 0.9980249896695866
Missing rate 0.1
train_X_fake diff: 14866296.0
valid_X_fake diff: 1653692.0
Start of epoch 0
Training loss over epoch:  0 0.10284654
Training acc over epoch:  0 0.97650254
Validation loss:  0 0.0044880686
Validation acc:  0 0.99930507
epoch 0: loss = 0.10284654
Start of epoch 1
Training loss over epoch:  1 0.0053770645
Training acc over epoch:  1 0.99837357
Validation loss:  1 0.0026679933
Validation acc:  1 0.99949396
epoch 1: loss = 0.005377066
Start of epoch 2
Training loss over epoch:  2 0.0033411107
Training acc over epoch:  2 0.99878544
Validation loss:  2 0.0016074161
Validation acc:  2 0.9996675
epoch 2: loss = 0.0033411104
Start of epoch 3
Training loss over epoch:  3 0.002441156
Training acc over epoch:  3 0.99907887
Validation loss:  3 0.0012001385
Validation a

100%|██████████| 1463/1463 [04:31<00:00,  5.38it/s]


The average imputation accuracyon test data with 0.1 missing genotypes is 0.9980: 
Sensitivity: 0.9980481585307146
Specificity: 0.9980481585307146
F1-score macro: 0.9980499090274304
F1-score micro: 0.9980499376784207
Missing rate 0.2
train_X_fake diff: 29732602.0
valid_X_fake diff: 3307384.0
Start of epoch 0
Training loss over epoch:  0 0.10427275
Training acc over epoch:  0 0.9730809
Validation loss:  0 0.0052408446
Validation acc:  0 0.99888957
epoch 0: loss = 0.10427275
Start of epoch 1
Training loss over epoch:  1 0.0064668353
Training acc over epoch:  1 0.99815
Validation loss:  1 0.0033793566
Validation acc:  1 0.99912244
epoch 1: loss = 0.0064668357
Start of epoch 2
Training loss over epoch:  2 0.0038029703
Training acc over epoch:  2 0.9986205
Validation loss:  2 0.0021063415
Validation acc:  2 0.9993941
epoch 2: loss = 0.0038029703
Start of epoch 3
Training loss over epoch:  3 0.002967526
Training acc over epoch:  3 0.99889064
Validation loss:  3 0.001811289
Validation acc:  3

100%|██████████| 1463/1463 [04:42<00:00,  5.17it/s]


The average imputation accuracyon test data with 0.2 missing genotypes is 0.9979: 
Sensitivity: 0.9979396984734099
Specificity: 0.9979396984734099
F1-score macro: 0.9979379012299723
F1-score micro: 0.9979379138523455
